# Iterator

## O que é?

O padrão _iterator_ prove uma forma de acessar elementos de uma colecao sequencialmente sem expor a implementacao dessa colecao

## Por quê?

Muitas vezes temos colecoes que demandam conhecimento para garantir que o usuario navegue por todos seus elementos (arvores, graphs, dicionarios, ou qualquer colacao customizada que voce montar) e iterator funciona como uma forma simples de abstrair esse conhecimento da implementacao e permitir que se utilize diferentes colecoes da mesma forma

## Estrutura

![figura](./assets/iterator.gif)

## Exemplo

Vamos exemplificar o iterator patterns usando uma colecao de palavras (por simplicidade, essa colecao sera implementada com uma lista mas ela poderia ser uma sorted tree ou qualquer outra data structure)

Para iterar sobre as palavras dessa colecao fizemos um iterator que ira iterar em ordem alfabetica (ou reversa) sobre os elementos da colecao

Como a colecao eh responsavel por prover o iterator, o usuario nao precisa saber sobre como a navegacao entre elementos eh implementada. Nesse caso ordenamos a lista e mantemos o track do indice, se a estrutura fosse uma sorted tree a navegacao seria completamente diferente (https://en.wikipedia.org/wiki/Tree_traversal) mas do ponto de vista do usuario nada mudaria.

In [9]:
from __future__ import annotations
from collections.abc import Iterable, Iterator
from typing import Any, List


class AlphabeticalOrderIterator(Iterator):
    """
    Concrete Iterators implement various traversal algorithms. These classes
    store the current traversal position at all times.
    """

    """
    `_position` attribute stores the current traversal position. An iterator may
    have a lot of other fields for storing iteration state, especially when it
    is supposed to work with a particular kind of collection.
    """
    _position: int = None

    """
    This attribute indicates the traversal direction.
    """
    _reverse: bool = False

    def __init__(self, collection: WordsCollection, reverse: bool = False) -> None:
        collection.sort()
        self._collection = collection
        self._reverse = reverse
        self._position = -1 if reverse else 0

    def __next__(self):
        """
        The __next__() method must return the next item in the sequence. On
        reaching the end, and in subsequent calls, it must raise StopIteration.
        """
        try:
            value = self._collection[self._position]
            self._position += -1 if self._reverse else 1
        except IndexError:
            raise StopIteration()

        return value


class WordsCollection(Iterable):
    """
    Concrete Collections provide one or several methods for retrieving fresh
    iterator instances, compatible with the collection class.
    """

    def __init__(self, collection: List[Any] = []) -> None:
        self._collection = collection

    def __iter__(self) -> AlphabeticalOrderIterator:
        """
        The __iter__() method returns the iterator object itself, by default we
        return the iterator in ascending order.
        """
        return AlphabeticalOrderIterator(self._collection)

    def get_reverse_iterator(self) -> AlphabeticalOrderIterator:
        return AlphabeticalOrderIterator(self._collection, True)

    def add_item(self, item: Any):
        self._collection.append(item)

collection = WordsCollection()
collection.add_item("Banana")
collection.add_item("Abacate")
collection.add_item("Caju")

print("Straight traversal:")
print("\n".join(collection))
print("")

print("Reverse traversal:")
print("\n".join(collection.get_reverse_iterator()), end="")
        

Straight traversal:
Abacate
Banana
Caju

Reverse traversal:
Caju
Banana
Abacate

## Pros e cons

__Pros__
- Iterators sao uma forma homogenea de navegar colecoes de dados independente de qual data structure esta por baixo da colecao
- Iterators permite que voce tenha diversas estrategias de navegacao para a mesma colecao, trees sao um bom exemplo de multiplas formas de navegacao para uma mesma estrutura alem disso voce pode implementar iterators para ordenar ou filtrar colecoes.
- Voce pode ter diversos iterators sobre a mesma colecao sem que um afete o outro 
- Iterators protegem contra "mutation" (quando voce passa a referencia de uma data structure e ela (a original) acaba sendo alterada inadivertidamente)
- Com iterators voce pode ter colecoes infinitas (na pratica o iterator gera os valores, ao invez de ler em uma data structure) ex: um random number generator

__Cons__
- O iterator eh coupled com a colecao entao voce nao consegue reutilizar o mesmo iterator para diversas colecoes
- Para colecoes simples (array, linked lists) um decorator (que nao faca nada alem de iterar tipo sort ou filter) nao eh necessario
- Esse pattern eh tao bom, que eh implementado na maioria das linguas, entao fica dificil pensar em cons

## Possíveis _use cases_

- Esse pattern eh a forma padrao de iterar sobre arrays, maps, linked lists na maioria das linguagens modernas
- Iterators sao uma forma pratica de filtrar e ordenar colecoes

## Perguntas
- Consider a composite that contains loan objects. The loan object interface contains a method called "AmountOfLoan()", which returns the current market value of a loan. Given a requirement to extract all loans above, below or in between a certain amount, would you write or use an Iterator to do this?